In [13]:
import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_columns = None
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [14]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Assuming df is your DataFrame and 'Feature17' has missing values
imputer = IterativeImputer(max_iter=100, tol=1e-3)

df_train = pd.read_csv('../data/iith_foml_2023_train.csv')
# Fit the imputer model on the dataset to learn the data patterns
df_train.drop(columns=['Feature 16', 'Feature 17'], inplace=True)
imputer.fit(df_train)

# Transform the dataset to replace missing values
df_imputed = imputer.transform(df_train)

# Convert back to a DataFrame
df_imputed = pd.DataFrame(df_imputed, columns=df_train.columns)

In [15]:
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight

X_train = df_imputed.iloc[:, :-1]
y_train = df_imputed.iloc[:, -1]
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
class_wts = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_wts_dict = dict(zip(np.arange(len(class_wts)), class_wts))
sample_wts = compute_sample_weight(class_weight=class_wts_dict, y=y_train)

In [16]:
from sklearn.ensemble import VotingClassifier, BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, classification_report, make_scorer
from xgboost import XGBClassifier

def report_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro', zero_division=0.0)

# Initialize models
extratrees_model = ExtraTreesClassifier(random_state=42)
# bagging_xgboost_model = BaggingClassifier(estimator=XGBClassifier(random_state=42), random_state=42)
randomforest_model = RandomForestClassifier(random_state=42)
graidentboost_model = GradientBoostingClassifier(random_state=42, n_estimators=200)

model = VotingClassifier(
    estimators=[
        ('bagging_extratrees', extratrees_model),
        # ('bagging_xgboost', bagging_xgboost_model),
        ('bagging_gradboost', randomforest_model),
        ('gradientboost_model', graidentboost_model)
    ], voting='hard'
)

# Train the model on the training data
cv_f1_score = cross_val_score(model, X_train, y_train, scoring=make_scorer(report_f1))
print(cv_f1_score)

/home/goatsadmin/.pyvenv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[0.44036103 0.47374463 0.39088301 0.46642407 0.51430373]
